# COMP9318 Lab2

## Instructions
1. This note book contains instructions for **COMP9318-Lab2**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures return by corresponding functions.

* You need to submit the code for **lab2** via following link: https://kg.cse.unsw.edu.au/submit/

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* If you choose to skip a question, leave the corresponding function body as it is (i.e., keep the `pass` line), otherwise it may affect your mark for other questions.

* You are allowed to add other functions and/or import additional modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **23:59:59 on 21st March, 2019**. We will **not** accept any late submissions.

# Question 1: Optimal binning algorithm using dynamic programming (100 points)

In [3]:
# These are the only modules that you can use in lab2
import pandas as pd
import numpy as np

In this lab, you need to implement the optimal binning algorithm using the dynamic programming algorithm we discussed in the lecture. You are only allowed to use $O(n^2)$ space. 

## Input

The input contains data (in a list) and the number of bins (an integer).

## Output

You are required to output the binning result and the matrix computed by the algorithm.

The matrix entries record optimal binning cost for a suffix of the input array using a certain number of bins. You should assign -1 to all the invalid solutions.

In [4]:
x=[3, 1, 18, 11, 13, 17]
num_bins = 4

In [5]:
def get_sse(l):
    if len(l)==0:
        return 0.0
    # average of the list
    average = np.average(l)
    ll =[]
    for i in l:
        x = (i-average)*(i-average)
        ll.append(x)
    value = sum(ll)
    return value

def v_opt_dp2(x, b, matrix, matrix_list,matrix_list_best ,x_leave, bin_empty):
    if (x_leave == len(x)):
        #print('b')
        x_leave = 0
        bin_empty = bin_empty - 1
        if(bin_empty <0):
            return
        v_opt_dp2(x, b, matrix, matrix_list, matrix_list_best, x_leave, bin_empty)
    if bin_empty <=x_leave and (len(x) - x_leave) >= (b - bin_empty):
        rest_list = x[x_leave:]
        #print(rest_list)
        #print(bin_empty)
        #print(x_leave)
        if bin_empty == b-1:
            cost = get_sse(rest_list)
            matrix[0][x_leave] = cost
            matrix_list[0][x_leave]= [rest_list]
            matrix_list_best[0][x_leave] = [rest_list]
            v_opt_dp2(x, b, matrix, matrix_list,matrix_list_best ,x_leave + 1, bin_empty)
        else:
            currcostL = []
            pathL =[]
            if len(rest_list) == b - bin_empty:
                min_cost = 0
                for e in rest_list:
                    pathL.append([e])
            else:
                for h in range(1,len(rest_list)-(b-bin_empty)+2):
                    prefix = rest_list[:h]
                    suffix = rest_list[h:]
                    pre_cost = get_sse(prefix)
                    # serch the previous result as index
                    index = matrix_list[b-bin_empty-2].index([suffix])
                    # print('index:',index)
                    suf_path = matrix_list_best[b-bin_empty - 2][index]
                    suf_cost = matrix[b-bin_empty - 2][index]

                    cost = pre_cost + suf_cost
                    #print(cost)
                    currcostL.append(cost)

                    curr_path = [prefix] + suf_path
                    pathL.append(curr_path)
                # select the best result(min cost) to put in the matrix
                min_cost = min(currcostL)
                # put the min cost list into the best_matrix_list
                min_cost_index = currcostL.index(min_cost)
                pathL = pathL[min_cost_index]
            # record the best list into best_matrix_list
            #print('b',b)
            #print('bin_empty', bin_empty)
            #print('b-bin_empty', b-bin_empty)
            matrix_list_best[b-bin_empty - 1][x_leave] = pathL
            matrix[b-bin_empty - 1][x_leave] = min_cost
            matrix_list[b-bin_empty - 1][x_leave] = [rest_list]

            v_opt_dp2(x, b, matrix, matrix_list, matrix_list_best, x_leave+1, bin_empty)
    else:
        #print('z')
        v_opt_dp2(x, b, matrix, matrix_list, matrix_list_best, x_leave + 1, bin_empty)



def v_opt_dp(x, b):
    # record the best cost
    matrix = [[-1 for i in range(len(x))] for j in range(b)]
    # record index
    matrix_list = [[0 for i in range(len(x))] for j in range(b)]
    matrix_list_best = [[0 for i in range(len(x))] for j in range(b)]
    x_leave = 0
    bin_empty = b - 1
    v_opt_dp2(x, b, matrix, matrix_list,matrix_list_best ,x_leave, bin_empty)
    bins = matrix_list_best[-1][0]
    return matrix,bins


In [6]:
## You can test your implementation using the following code...

import submission as submission
matrix, bins = submission.v_opt_dp(x, num_bins)
print("Bins = {}".format(bins))
print("Matrix =")
for row in matrix:
    print(row)

Bins = [[3, 1], [18], [11, 13], [17]]
Matrix =
[-1, -1, -1, 18.666666666666664, 8.0, 0.0]
[-1, -1, 18.666666666666664, 2.0, 0, -1]
[-1, 18.666666666666664, 2.0, 0, -1, -1]
[4.0, 2.0, 0, -1, -1, -1]
